# Исследование датасета опиоидных рецепторов


In [1]:
!pip install chembl_webresource_client rdkit pandas dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.2/34.2 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [2]:
# Импортируем нужные библиотеки
import pandas as pd
import numpy as np
import seaborn
import matplotlib.pyplot as plt
from scipy import stats as st
from rdkit import Chem
from rdkit.Chem import AllChem
import sqlite3
from chembl_webresource_client.new_client import new_client
from rdkit.DataStructs.cDataStructs import ConvertToNumpyArray
from rdkit.Chem import Descriptors, MolFromSmiles
import sqlite3
import dash
from dash import dcc, html
import plotly.express as px
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt
import seaborn as sns
colors = ['#082040', '#175073', '#3285A6', '#B8D0D9', '#6CC5D9']

## 1. Загрузка датасетов
Опиоидных рецепторов пять, поэтому загрузим все датасеты, сольем их в один и добавим столбец, в котором будет содержаться информация из какого датасета иначально взяты данные.
Для данной задачи эффективнее использовать наработки из готовых БД, чем использовать парсинг данных. Используем бд ChEMBL для поиска данных.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##  2. Статистический анализ данных

В качестве метрик качества датасета будем использовать различные статистические тесты.

In [4]:
df_ki_original = pd.read_csv('/content/drive/MyDrive/Opioid_receptor/df_ki_original_descriptors.csv', sep = ',', low_memory=False)

### 2.1 Анализ распределения целевой величины

In [5]:
fig = make_subplots(rows=1, cols=2, subplot_titles=['Гистограмма', 'Violin Plot'])

# Гистограмма
hist_fig = px.histogram(df_ki_original, x="pChEMBL_Value", nbins = 60,
                 color_discrete_sequence = colors,
                 opacity = 0.7)

fig.add_trace(hist_fig['data'][0], row=1, col=1) # Добавление графика с указанием расположения

# Violin plot
violin_fig = px.violin(df_ki_original, y="pChEMBL_Value", color_discrete_sequence = colors, box = True)
fig.add_trace(violin_fig['data'][0], row=1, col=2) # Добавление графика с указанием расположения

# Настройка макета
fig.update_layout(showlegend=False, title_text="Распределение pChEMBL_Value")

# Отображение графика
fig.show()

Не распределение - а мечта по целевой переменной pChEMBL_Value (pKi)

**Вывод:** В целом распределение похоже на нормальное, однако наблюдается несколько пиков, мы наблюдаем некоторое количество выбросов, с которыми будем дальше работать.

**Статистический тест на нормальность распределения**

На этом этапе мы можем провести статистический тест, чтобы узнать нормальное ли у нас распределение. Для этого используем тест Андерсона-Дарлинга

In [6]:
# Проведем тест Андерсона-Дарлинга на нормальность
#В параметре dist указываем необходимое нам распределение - нормальное
result = st.anderson(df_ki_original['pChEMBL_Value'], dist='norm')

#Выводи весь результат теста
print(f"Результат теста: {result}")

# Результат теста будет содержать статистику и критические значения
print('Статистика теста:', result.statistic)
print('Критические значения:', result.critical_values)

# Оценка уровня значимости на основе статистики теста, статистики в тесте считаются для конкретных уровней значимости
print('Уровень значимости:', result.significance_level)

# Оценим результат теста на нормальность
if result.statistic < result.critical_values[2]: #Данная статистика считается для уровня значимости 0.05 (5%)
    print("Данные похожи на нормальное распределение (гипотеза о нормальности не отвергается).")
else:
    print("Данные не похожи на нормальное распределение (гипотеза о нормальности отвергается).")

Результат теста: AndersonResult(statistic=18.04310156874635, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=-1.712275821619136, scale=1.3366343584391096)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')
Статистика теста: 18.04310156874635
Критические значения: [0.576 0.656 0.787 0.918 1.092]
Уровень значимости: [15.  10.   5.   2.5  1. ]
Данные не похожи на нормальное распределение (гипотеза о нормальности отвергается).


Наши данные не соответвуют нормальному распределению, что нужно учитывать при дальнейшем анализе данных и обучении моделей.

Я объединила все данные  из 4 групп, возможно, что наличие выбросов обусловлено тем, что какая-то группа молекул имеет сильно отличающиеся от других свойства. Поэтому на данном этапе построим гистограммы и Violin Plot для различных групп.

In [7]:
# Отсортируем данные по разным группам для более наглядной визуализации
df_sort_group = df_ki_original.sort_values(by='Target')

In [8]:
# Гистограмма
hist_fig_diff = px.histogram(df_sort_group, x= 'pChEMBL_Value', color = 'Target', nbins = 60,
                 color_discrete_sequence = colors,
                 opacity = 0.7)

hist_fig_diff.show()

In [9]:
# Violin plot
violin_fig_diff = px.violin(df_sort_group, y='pChEMBL_Value', color = 'Target', color_discrete_sequence = colors, box = True)

# Отображение графика
violin_fig_diff.show()

**Вывод:** Рассмотрев данные гистограммы и violin plot, мы можем предположить, что распределение зависит от отношения молекулы к определенной группе

Мы можем это проверить на статистическом тесте.

**Статистический тест**

Используем тест Крускала-Уоллиса, чтобы проверить, есть ли статистически значимые различия между группами. Мы выбрали непараметрический тест, так как наше распределение не является нормальным.

In [10]:
# Проведем тест Крускала-Уоллиса
statistic, p_value = st.kruskal(df_ki_original[df_ki_original['Target'] == 'Mu']['pChEMBL_Value'],
                                      df_ki_original[df_ki_original['Target'] == 'Kappa']['pChEMBL_Value'],
                                      df_ki_original[df_ki_original['Target'] == 'Delta']['pChEMBL_Value'],
                                      df_ki_original[df_ki_original['Target'] == 'Nociceptin']['pChEMBL_Value'])

# Вывод результатов
print("Значение статистики:", statistic)
print("Значение p-значения:", p_value)

# Оценка статистической значимости
if p_value > 0.05:  # Уровень значимости
    print("Нет статистически значимой разницы между выборками")
else:
    print("Существует статистически значимая разница между выборками")

Значение статистики: 265.7282390986002
Значение p-значения: 2.5919712601955436e-57
Существует статистически значимая разница между выборками


Мы выяснили, что есть статистическая разница между группами, что нам нужно будет учитывать в дальнейшем работе, то есть обучение моделей стоит проводить не на всем датасете, а отдельно для каждой группы по Target.

### 2.2 Анализ распределения непрерывных переменных

In [11]:

targets = ['Mu', 'Kappa', 'Delta', 'Nociceptin']

# Подсчет строк для каждой цели
target_counts = {target: df_ki_original[df_ki_original['Target'] == target].shape[0] for target in targets}

# Подсчет уникальных молекул для всех пересечений
intersection_counts = {}
k = 0
for target in targets:
    intersection_df = df_ki_original[df_ki_original['Target'] == target]
    unique_mols_count = intersection_df['Mol'].nunique()
    intersection_counts[target] = unique_mols_count
    k += unique_mols_count

# Вывод результатов
print("\nКоличество уникальных Mol для каждого Target:")
for target, unique_count in intersection_counts.items():
    print(f"{target}: {unique_count}")
print(k)
print(len(df_ki_original))


Количество уникальных Mol для каждого Target:
Mu: 4299
Kappa: 3636
Delta: 3632
Nociceptin: 1160
12727
12727


In [ ]:
len(set(df_ki_original['Mol']))

12727

In [ ]:
# Фильтрация по целям
filtered_df = df_ki_original[(df_ki_original['Target'] == 'Delta') | (df_ki_original['Target'] == 'Nociceptin')]

# Подсчет уникальных молекул, которые принадлежат либо Delta, либо Nociceptin
unique_mols = filtered_df['Mol'].nunique()

# Вывод результата
print(f"Количество уникальных молекул, которые принадлежат либо Nociceptin, либо Delta: {unique_mols}")

Количество уникальных молекул, которые принадлежат либо Nociceptin, либо Delta: 4792


In [ ]:
# Фильтрация по целям
filtered_df = df_ki_original[(df_ki_original['Target'] == 'Mu') | (df_ki_original['Target'] == 'Kappa')]

# Подсчет уникальных молекул, которые одновременно принадлежат Mu и Kappa
unique_mols = filtered_df.groupby('Mol').filter(lambda x: len(x) > 1)['Mol'].nunique()

# Вывод результата
print(f"Количество уникальных молекул, которые одновременно принадлежат Mu и Kappa: {unique_mols}")

Количество уникальных молекул, которые одновременно принадлежат Mu и Kappa: 0


In [ ]:
# Фильтрация по целям
filtered_df = df_ki_original[(df_ki_original['Target'] == 'Mu') | (df_ki_original['Target'] == 'Delta')]

# Подсчет уникальных молекул, которые одновременно принадлежат Mu и 'Delta'
unique_mols = filtered_df.groupby('Mol').filter(lambda x: len(x) > 1)['Mol'].nunique()

# Вывод результата
print(f"Количество уникальных молекул, которые одновременно принадлежат Mu и Delta: {unique_mols}")

Количество уникальных молекул, которые одновременно принадлежат Mu и Delta: 0


In [ ]:
# Фильтрация по целям
filtered_df = df_ki_original[(df_ki_original['Target'] == 'Mu') | (df_ki_original['Target'] == 'Nociceptin')]

# Подсчет уникальных молекул, которые одновременно принадлежат Mu и 'Delta'
unique_mols = filtered_df.groupby('Mol').filter(lambda x: len(x) > 1)['Mol'].nunique()

# Вывод результата
print(f"Количество уникальных молекул, которые одновременно принадлежат Mu и Nociceptin: {unique_mols}")

Количество уникальных молекул, которые одновременно принадлежат Mu и Nociceptin: 0


In [ ]:
# Фильтрация по целям
filtered_df = df_ki_original[(df_ki_original['Target'] == 'Kappa') | (df_ki_original['Target'] == 'Delta')]

# Подсчет уникальных молекул, которые одновременно принадлежат Mu и 'Delta'
unique_mols = filtered_df.groupby('Mol').filter(lambda x: len(x) > 1)['Mol'].nunique()

# Вывод результата
print(f"Количество уникальных молекул, которые одновременно принадлежат Kappa и Delta: {unique_mols}")

Количество уникальных молекул, которые одновременно принадлежат Kappa и Delta: 0


In [ ]:
# Фильтрация по целям
filtered_df = df_ki_original[(df_ki_original['Target'] == 'Kappa') | (df_ki_original['Target'] == 'Nociceptin')]

# Подсчет уникальных молекул, которые одновременно принадлежат Mu и 'Delta'
unique_mols = filtered_df.groupby('Mol').filter(lambda x: len(x) > 1)['Mol'].nunique()

# Вывод результата
print(f"Количество уникальных молекул, которые одновременно принадлежат Kappa и Nociceptin: {unique_mols}")

Количество уникальных молекул, которые одновременно принадлежат Kappa и Nociceptin: 0


In [ ]:
# Фильтрация по целям
filtered_df = df_ki_original[(df_ki_original['Target'] == 'Delta') | (df_ki_original['Target'] == 'Nociceptin')]

# Подсчет уникальных молекул, которые одновременно принадлежат Mu и 'Delta'
unique_mols = filtered_df.groupby('Mol').filter(lambda x: len(x) > 1)['Mol'].nunique()

# Вывод результата
print(f"Количество уникальных молекул, которые одновременно принадлежат Nociceptin и Delta: {unique_mols}")

Количество уникальных молекул, которые одновременно принадлежат Nociceptin и Delta: 0


## 3. Исследовательский анализ данных

Проведем исследовательский анализ данных. Посмотрим, какие структурные фрагменты наиболее часто встречаются в базе данных, какие фрагменты наиболее характерны для самых активных и самых неактивных молекул.

In [12]:
# Вывод топ-200 строк с максимальными значениями по pChEMBL_Value
top_200_max = df_ki_original.nlargest(200, 'pChEMBL_Value')

# Вывод топ-200 строк с минимальными значениями по pChEMBL_Value
top_200_min = df_ki_original.nsmallest(200, 'pChEMBL_Value')

In [13]:
def get_nonul(df, s):
    # Предположим, что top_200_min уже определен
    # Выбор столбцов с индексами от 120 до 219
    df_m = df.iloc[:, 120:219]
    df_m['Target'] = df['Target']
    # Шаг 1: Определить колонки с пустыми или равными нулю значениями
    empty_or_zero_columns = df_m.columns[(df_m.isnull().all() | (df_m == 0).all())].tolist()

    # Шаг 2: Определить колонки с процентом ненулевых значений больше 20%
    non_zero_percentage = (df_m != 0).mean() * 100
    high_percentage_columns = non_zero_percentage[non_zero_percentage > 20].index.tolist()
    high_percentage_values = non_zero_percentage[non_zero_percentage > 20]

    # Подсчет общего количества строк
    total_rows = df_m.shape[0]

    # Создание DataFrame для хранения процентов по Target для каждого дескриптора
    results = []

    # Перебор каждого дескриптора в high_percentage_columns
    for descriptor in high_percentage_columns:
        # Фильтрация DataFrame по ненулевым значениям в текущем дескрипторе
        filtered_df = df_m[df_m[descriptor].notnull() & (df_m[descriptor] != 0)]

        # Подсчет количества строк по Target для текущего дескриптора
        target_counts = filtered_df['Target'].value_counts()

        # Вычисление процента для каждого значения Target
        target_percentages = (target_counts / total_rows) * 100

        # Добавление результатов в список
        for target, percentage in zip(target_counts.index, target_percentages):
            results.append({'Descriptor': descriptor, 'Target': target, 'Percentage': percentage})

    # Преобразование результатов в DataFrame
    results_df = pd.DataFrame(results)
    # Создание Dash приложения
    app = dash.Dash(__name__)

    # Создание графика
    fig_m = px.bar(results_df,
                  x='Descriptor',
                  y='Percentage',
                  color='Target',
                  title=f'Процент ненулевых значений по дескрипторам для {s}',
                  labels={'Descriptor': 'Дескрипторы', 'Percentage': 'Процент (%)'},
                  barmode='stack')

    fig_m.add_hline(y=20, line_dash="dash", line_color="red", annotation_text="20% Threshold", annotation_position="top right")

    return fig_m

In [14]:
fig_all  = get_nonul(df_ki_original, 'полного датасета')
fig_max = get_nonul(top_200_max, 'топ-200 самых активных молекул')
fig_min = get_nonul(top_200_min, 'топ-200 самых неактивных молекул')

Проанализируем распределения QED и MolWt

In [15]:
fig_qed_molwt = make_subplots(rows=1, cols=2, subplot_titles=['Распределение QED', 'Violin Plot MolWt'])

# Гистограмма
hist_fig_qed = px.histogram(df_ki_original, x="qed", nbins = 60,
                 color_discrete_sequence = colors,
                 opacity = 0.7)

fig_qed_molwt.add_trace(hist_fig_qed['data'][0], row=1, col=1) # Добавление графика с указанием расположения

# Violin plot
violin_fig_molwt = px.violin(df_ki_original, y="qed", color_discrete_sequence = colors, box = True)
fig_qed_molwt.add_trace(violin_fig_molwt['data'][0], row=1, col=2) # Добавление графика с указанием расположения

# Настройка макета
fig_qed_molwt.update_layout(showlegend=False, title_text="Гистограмма и Violin Plot")

# Отображение графика
fig_qed_molwt.show()

## 4. Создание общего дашборда

In [16]:
# Статистический анализ и визуализация данных с помощью Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='combined-plot', figure=fig),  # Используем созданный график
    dcc.Graph(id='hist', figure=hist_fig_diff),  # Используем созданный график
    dcc.Graph(id='violin-plot', figure=violin_fig_diff),  # Используем созданный график
    dcc.Graph(
        id='bar-chart-1',
        figure=fig_all
    ),

    # Второй график (можно заменить на fig2 если у вас есть другой набор данных)
    dcc.Graph(
        id='bar-chart-2',
        figure=fig_max
    ),
        # Второй график (можно заменить на fig2 если у вас есть другой набор данных)
    dcc.Graph(
        id='bar-chart-3',
        figure=fig_min
    ),
    dcc.Graph(id='qed_molwt', figure=fig_qed_molwt),
    dcc.Graph(
        id='descriptor-graph',
        figure=px.scatter(df_ki_original, x='qed', y='pChEMBL_Value', color='Target',
                          title='Зависимость pChEMBL_Value от QED')
    )
])


if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>